# Reinforcement Learning in Action
## Course1: Introduction to Reinforcement Learning
___

## Course Syllabus
---
- What is RL?
- What kind of problems does RL solve?
- **C1: Foolsball**
- Modeling problems using the RL framework: Agent, Environment, State, Goals, Rewards, Returns
- MDPs and single-step dynamics
- State-values, action values, policies, optimality
- Solving MDP with known single-step dynamics
- Monte Carlo estimation, greedy policies, exploration and exploitation, epsilon-greedy policies
- TD methods: Sarsa, Sarsamax Q-learning, Expected Sarsa
- **P1: Taxi-V3**
- Discretization: Tile-coding, Kernels

## Week 3 Summary
---

## The exploration-exploitation dilemma

### Exploration using random sampling

Select a uniformly random action in every state within an episode

Good strategy to learn about the environment in the beginning

In [ ]:
action = np.random.choice(foolsball.actions)

### Exploitation using a greedy sampling

Select the action corresponding to the highest discounted return in every state within an episode

Does not give good results in the beginning when the returns table is based on very little experience

In [ ]:
greedy_action = table.loc[state].idxmax()

### Combining Exploration and Exploitation  with $\epsilon$-greedy sampling

Select the action with the highest discounted return most of the time

Select a random action with a small probability

In [ ]:
actions = table.columns
action_probs = np.asarray([epsilon/len(actions)]*len(actions),dtype=np.float)

greedy_action_index = np.argmax(table.loc[state].values)
action_probs[greedy_action_index] += 1-epsilon

epsilon_greedy_action = np.random.choice(table.columns,p=action_probs)

### $\epsilon$ Decay

Start with pure exploration

Rely more and more on exploitation with each passing episode

In [ ]:
episode_i = collect_epsilon_greedy_episode_from_returns_tbl(estimated_returns,epsilon=epsilon)
epsilon *= epsilon_decay
epsilon = max(epsilon, min_epsilon)

## Converging Faster

### Constant Alpha ($\alpha$)

The division by number of visits has a non-linear effect on the size of every new update to the returns table

Use the difference of existing estimate and new estimate based on an episode to update the existing estimate 

In [ ]:
ESTIMATED_RETURNS_TBL.loc[s,a] += alpha*(ret - ESTIMATED_RETURNS_TBL.loc[s,a])

### Temporal Difference (TD) Learning

Update the returns table after every step of an episode 

In the absence of an entire episode, use the difference of the current estimate and new estimate based on just on step

In [ ]:
ESTIMATED_RETURNS_TBL.loc[s0,a0] +=\
    alpha*(reward + HYPER_PARAMS['gamma']*ESTIMATED_RETURNS_TBL.loc[s1,a1] - ESTIMATED_RETURNS_TBL.loc[s0,a0])

### SARSA

```
for i in tqdm(range(n_episodes)):
    foolsball.reset()
    s0 = foolsball.init_state
    a0 = epsilon_greedy_action_from_Q(Q,s0,epsilon)
    done = False
    
    while not done:
        s1, reward, done  = foolsball.step(a0)
        a1 = epsilon_greedy_action_from_Q(Q,s1,epsilon)
        
        Q.loc[s0,a0] += alpha*(reward + HYPER_PARAMS['gamma']*Q.loc[s1,a1] - Q.loc[s0,a0])
        
        s0, a0 = s1, a1
  
    epsilon *= epsilon_decay
    epsilon = max(epsilon,min_epsilon)

```

### Q-Learning

```
for i in tqdm(range(n_episodes)):
    foolsball.reset()
    s0 = foolsball.init_state
    done = False
    
    while not done:
        a0 = epsilon_greedy_action_from_Q(Q,s0,epsilon)
        s1, reward, done  = foolsball.step(a0)
        
        Q.loc[s0,a0] += alpha*(reward + HYPER_PARAMS['gamma']*Q.loc[s1].max() - Q.loc[s0,a0])
        
        s0 = s1
  
    epsilon *= epsilon_decay
    epsilon = max(epsilon,min_epsilon)
```

## Terminology
---

### Environment and Agent

Many problems requiring **learning** to plan and control can be modeled as an agent and environment set up

![](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTMDmrmnl_dAyjCOErHPak2gLXmQTgQnVT8gQ&usqp=CAU)

- The agent performs an action in the environment
- The state of the environment and agent change as a result
- The agent receives a reward and the updated state from the environment
- The agent and environment distinction is a little tricky to define 

**Sutton & Barto**
> The agent–environment boundary can be located at different places for different purposes.

### State and Actions

- The state is a description of the environment and the agent, that is used for decision making
- The decisions that the agent makes are its actions and they impact the state of the system 
- Technically state is the precise desciption of the system while observations are incomplete/noisy state information [\[1\]](https://ai.stackexchange.com/a/5971/23273)

### Markov Decision Processes (MDP)

- A markov decision process is a state transition model [\[1\]](https://setosa.io/ev/markov-chains/) that takes an action in every state and produces a reward along with any state transition.
- They're Markovian because the transition probabilities in any state only depend on the current state and not on the previous states that led to the current state.

- An MDP is made up of the following
    - A set $S$ representing all the states
    - A set $A$ representing all possible actions 
    - A transition matrix P that maps (state,action) pairs to next states: $P:S \times A \rightarrow S$
    - A reward matirx R that maps (state,next state) pairs to immediate rewards(real numbers): $R:S \times S \rightarrow \mathbb{R}$

When the environment is stochastic there could be multiple possible next states for every state action pair. The transition table is then three dimensional 
$$P: S \times A \times S \rightarrow [0,1]$$

### Single-Step Dynamics

- The transition annd rewards tables are collectively known as the single-step dynamics
- The dynamics are generally not known to the agent 

### Rewards, Returns and Discounted Returns

- The immediate reward that follows a state transition is denoted by $R$.
- The cumulative reward from a state to a terminal state is denoted by $G$ and called return.
- The cumulative reward is often discounted when training an agent to nudge the agent from looking too far in the future.

### Policies

- A deterministic policy maps every state to an action
$$ \pi : S \rightarrow A$$

- An RL algorithm's goal is to find an optimal policy $\pi^*$ that maps every state to the action with the highest discounted return 

### State Value and Action Value functions

- The state value function maps each state to the estimated discounted return for that state when following a policy $\pi$
$$V_{\pi}: S \rightarrow \mathbb{R}$$

- The action value function maps each $(state, action)$ pair to the estimated discounted return correspoding to the pair when following a policy $\pi$
$$Q_{\pi}: S \times A \rightarrow \mathbb{R}$$

### Monte Carlo Estimation 

- Monte Carlo methods are algorithms that use random sampling to find numerical solutions to problems [\[1\]](https://www.scratchapixel.com/lessons/mathematics-physics-for-computer-graphics/monte-carlo-methods-in-practice/monte-carlo-methods), [\[2\]](https://brilliant.org/wiki/monte-carlo/).

- Monte Carlo estimation finds the single step dynamics by sampling the environment through interaction.

### First Visit vs Every Visit

- When calculating returns for a $(state,action)$ pair from rewards estimated using Monte Carlo sampling
    - First visit estimation considers all visits to the $(state,action)$ pair in an episode.
    - Every visit estimation considers all visits to the $(state,action)$ pair in an episode.

### Greedy Policies

- A greedy policy selects the optimal action (action with the highest discounted return) in every state
- This requires use of the action value function $Q_{\pi}$
- Since action values are discounted returns and thus include future rewards,the greedily-selected actions are optimal

### $\epsilon$-Greedy Policies 

- An epsilon greedy policy selects the optimal action with a high probability but can also select a non-optimal action with a small probability 

- The hyper-paramtere $\epsilon$ controls the probability of selecting a non-optimal action

### Optimal State and Action Value Functions

- An optimal action value function $Q^*_{\pi}$ is the maximum action value function over all possible policies.
- An optimal state value function $V^*_{\pi}$ is the maximum state value function over all possible policies.

### Optimal Policy

- An optimal policy $\pi^*$ is a (greedy) policy derived from an optimal action value function

### Policy Evaluation

- Policy evaluation is the process of using a policy to estimate the state value function $V_{\pi}$corresponding to that policy



### Policy Improvement

- Policy improvement is the process of updating the action value function $Q_{\pi}$ and ultimately the policy $\pi$ based on the state value function $V_{\pi}$.


### Bellman Equations

TBD after introduction to stochastic policies and environments.

### Exploration vs Exploitation

### Epsilon Decay